<!-- quarto render updates.qmd --to html -->

## Motivational Example

Given seismic observations of a CO2 plume, can we invert for the underlying permeability distribution?

| $p(K|d) \propto p(d|K)\cdot p(K)$
|
|   $K$ is our permeability
|   $d$ is our seismic observation

Since $p(d|K)$ is expensive to compute, we train a FNO as a surrogate to offload some of the computation.

## Background of Fourier Neural Operators (FNOs)

Learns approximations to the solution operator to parametric PDEs

![FNO Architechture](assets/fno_arch.png){width=80%}

- Acts on functions in lifted space
- Spectral convolution (i.e learned weighting on low-frequency modes)

## Problem

Mapping permeability to time-varying CO2 concentration governed by two-phase flow equations.

| $n_x \cdot n_y \cdot n_t \cdot n_c$| samples | modes | sconv blocks |
|:--|:-|:-|:-|
| 64 x 64 x 51 x 4 | 1000 | 8 | 4 |

: Dataset details

## 10/4/23

| - Trained for 300/500 epochs on the GPU
|   - The model seems to have stop learning around ~150 epochs
|   - Objective is $\frac{norm(y - fno(x))}{norm(y)}$
|   - Using AdamW as optimizer
| - The weight application is on CPU (using third party library for einsum)

| **Currently working on**
|   - Debugging Kronecker Distribution
|   - Moving Einsum Computation to GPU (Currently ~ 4.5 days projected for 500 epochs)

#### Evaluation
::: {layout-ncol=2}
![](assets/10.4.23/300_eval.png)

![](assets/10.4.23/300_loss.png)
:::

## 10/13/23

| - Fixed Kronecker Distribution Bug
| - Made custom operators for Distrbuted Batch Norm, Distributed Weights Application
| - Trained serial FNO for 85 epochs
|   - Mapped the weights to a distributed FNO. Working `v1` forward pass

| **Currently working on** 
|   - Gradient for the network

| **Roadblocks** 
|  - No stable Einsum Package for Einsum on GPU
|    - `Einsum` : No GPU support
|    - `Tullio.jl` : GPU support but last release > 1 year, lots of errors setting up and no runtime contraction order support
|    - `OMEinsum.jl` : GPU support and runtime contraction support, failing for our use case on specific GPUs
|  - Mathias has suggested to open an issue with `OMEinsum.jl` Devs

#### Forward Pass DFNO on a 2x2 using CPU

![](assets/10.13.23/85_DFNO.png)

## 10/18/23

| - Added Operators and AD rules for gradient
| - Trained DFNO for 2 epochs

| **Currently working on**
|   - Code Cleanup
|   - Distributed IO
|   - GPU support

#### Trained DFNO on a 2x2 using CPU for 1 epoch

::: {layout-ncol=2}
![Serial](assets/10.18.23/2_eval.png){width=50%}

![](assets/10.18.23/2_loss.png){width=19%}
:::

::: {layout-ncol=2}
![Parallel](assets/10.18.23/dfno_2_eval.png){width=50%}

![](assets/10.18.23/dfno_2_loss.png){width=19%}
:::

## 10/30/23

| - Added Hacks for GPU Training of DFNO
| - Trained DFNO on 4 workers using GPU for 200 epochs
| - Porting 2D to 3D FNO

| **Currently working on**
|   - Gradient Tests
|   - Running jobs on Perlmutter

#### Trained 2D DFNO on a 2x2 using GPU for 200 epochs

::: {layout-ncol=2}
![](assets/10.30.23/ep%3D200_p%3D4_gpu%3D1_eval.png){width=50%}

![](assets/10.30.23/ep%3D200_p%3D4_gpu%3D1_loss.png){width=50%}
:::

#### Modular Codebase (~40 lines to train a DFNO)


In [ ]:
using Pkg
Pkg.activate("./")

include("src/models/DFNO_2D/DFNO_2D.jl")

using .DFNO_2D
using MPI

MPI.Init()

partition = [1,2,2,1]

comm = MPI.COMM_WORLD
@assert MPI.Comm_size(comm) == prod(partition)

modelConfig = DFNO_2D.ModelConfig(nx=64, ny=64, nt=51, nblocks=4, partition=partition)
dataConfig = DFNO_2D.DataConfig(modelConfig=modelConfig)

model = DFNO_2D.Model(modelConfig)
θ = DFNO_2D.initModel(model)

x_train, y_train, x_valid, y_valid = DFNO_2D.loadDistData(dataConfig)

trainConfig = DFNO_2D.TrainConfig(
    epochs=200,
    x_train=x_train,
    y_train=y_train,
    x_valid=x_valid,
    y_valid=y_valid,
)

DFNO_2D.train!(trainConfig, model, θ)

MPI.Finalize()